# Recommendation

# Step 1: Load the Dataset


In [10]:
import pandas as pd

# Load the dataset
file_path = 'recipe_final (1).csv'
recipe_df = pd.read_csv(file_path)

recipe_df.head()

,Unnamed: 0,recipe_id,recipe_name,aver_rate,image_url,review_nums,calories,fat,carbohydrates,protein,cholesterol,sodium,fiber,ingredients_list
5407,5407,12452,Cornish Pastie II,4.33,https://images.media-allrecipes.com/userphotos...,7,26,49,14,28,11,41,16,['recipe pastry for a (10 inch) double crust p...
1122,1122,20781,Saltimbocca alla Pollo,4.16,https://images.media-allrecipes.com/userphotos...,72,30,66,2,77,49,63,2,"['skinless boneless chicken breast halves', 'o..."
42990,42990,24931,Nutty Banana Bread,4.45,http://images.media-allrecipes.com/userphotos/...,24,11,15,10,7,13,8,8,"['all-purpose flour', 'baking soda', 'salt', '..."
36995,36995,245708,Crunchy Lemon Quinoa and Asparagus Bowl,3.57,https://images.media-allrecipes.com/userphotos...,5,8,6,8,26,0,1,35,"['fresh asparagus trimmed', 'coconut oil', 'on..."
26507,26507,23413,Dried Apple Stack Cake,3.50,http://images.media-allrecipes.com/userphotos/...,4,24,36,21,12,22,15,16,"['spice cake mix', 'dried apples', 'chopped pe..."


In [11]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [12]:
# Preprocess Ingredients
vectorizer = TfidfVectorizer()
X_ingredients = vectorizer.fit_transform(recipe_df['ingredients_list'])

In [13]:
# Normalize Numerical Features
scaler = StandardScaler()
X_numerical = scaler.fit_transform(recipe_df[['calories', 'fat', 'carbohydrates', 'protein', 'cholesterol', 'sodium', 'fiber']])

In [14]:
# Combine Features
X_combined = np.hstack([X_numerical, X_ingredients.toarray()])

# Train KNN Model
knn = NearestNeighbors(n_neighbors=3, metric='euclidean')
knn.fit(X_combined)

NearestNeighbors(metric='euclidean', n_neighbors=3)

In [15]:
# Function to Recommend Recipes
def recommend_recipes(input_features):
    input_features_scaled = scaler.transform([input_features[:7]])
    input_ingredients_transformed = vectorizer.transform([input_features[7]])
    input_combined = np.hstack([input_features_scaled, input_ingredients_transformed.toarray()])
    distances, indices = knn.kneighbors(input_combined)
    recommendations = recipe_df.iloc[indices[0]]
    return recommendations[['recipe_name', 'ingredients_list', 'image_url']]

# Example Input
input_features = [15, 36, 1, 42, 21, 81, 2, 'pork belly, smoked paprika, kosher salt']
recommendations = recommend_recipes(input_features)
recommendations

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,recipe_name,ingredients_list,image_url
0,Homemade Bacon,"['pork belly', 'smoked paprika', 'kosher salt'...",https://images.media-allrecipes.com/userphotos...
3507,Char Siu (Chinese Barbeque Pork),"['honey', 'Shaoxing cooking wine', 'hoisin sau...",https://images.media-allrecipes.com/userphotos...
9975,Pork Carnitas,"['vegetable oil', 'pork shoulder', 'kosher sal...",https://images.media-allrecipes.com/userphotos...
